# 03 — Entrenamiento y Evaluación (SUPERVISED y CONTINUAL con EWC)

Este notebook entrena un modelo SNN para **regresión de dirección (steering)** en:
- **Supervised**: circuito1
- **Continual**: `circuito1 → circuito2` con **EWC** (baseline)

Asegúrate de haber ejecutado antes `01_DATA_QC_PREP.ipynb` para generar los splits y `tasks.json`.


In [ ]:

# Imports y setup
from pathlib import Path
import sys, json, torch
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
# permite importar desde src/
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.utils import set_seeds, load_preset, make_loaders_from_csvs, ImageTransform
from src.models import SNNVisionRegressor
from src.training import TrainConfig, train_supervised, _permute_if_needed
from src.methods.ewc import EWC, EWCConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT, device


In [ ]:

# Selecciona preset
preset = "fast"   # 'fast' | 'std' | 'accurate'
cfg = load_preset(ROOT/"configs"/"presets.yaml", preset)
print("Preset:", cfg)

# Transformación de imagen (ajusta a tus necesidades si cambias el preprocesado)
tfm = ImageTransform(target_w=160, target_h=80, to_gray=True, crop_top=None)


In [ ]:

# Rutas de datos
RAW = ROOT/"data"/"raw"/"udacity"
PROC = ROOT/"data"/"processed"

# Comprobación rápida
for run in ["circuito1","circuito2"]:
    for part in ["train","val","test"]:
        path = PROC/run/f"{part}.csv"
        if not path.exists():
            raise FileNotFoundError(f"Falta {path}. Ejecuta 01_DATA_QC_PREP.ipynb primero.")
print("OK splits encontrados")


In [ ]:

# === SUPERVISED: circuito1 ===
set_seeds(42)
train_loader, val_loader, test_loader = make_loaders_from_csvs(
    base_dir=RAW/"circuito1",
    train_csv=PROC/"circuito1"/"train.csv",
    val_csv=PROC/"circuito1"/"val.csv",
    test_csv=PROC/"circuito1"/"test.csv",
    batch_size=cfg["batch_size"],
    encoder=cfg["encoder"],   # 'rate' o 'latency'
    T=cfg["T"],
    gain=cfg["gain"],
    tfm=tfm
)

model = SNNVisionRegressor(in_channels=1, lif_beta=0.95)
loss_fn = torch.nn.MSELoss()
tcfg = TrainConfig(epochs=cfg["epochs"], batch_size=cfg["batch_size"], lr=cfg["lr"], amp=cfg["amp"])

out_dir = ROOT/"outputs"/f"supervised_{preset}_ewc0"
print("Entrenando SUPERVISED...")
_ = train_supervised(model, train_loader, val_loader, loss_fn, tcfg, out_dir, method=None)
print("OK:", out_dir)


In [ ]:

# Helper para evaluar loaders asegurando la forma correcta (T,B,C,H,W)
def eval_loader(loader, model, device):
    mae_sum = mse_sum = 0.0
    n = 0
    for x, y in loader:
        x = _permute_if_needed(x.to(device))  # (B,T,C,H,W) -> (T,B,C,H,W)
        y = y.to(device)
        with torch.no_grad():
            y_hat = model(x)
        mae_sum += torch.mean(torch.abs(y_hat - y)).item() * len(y)
        mse_sum += torch.mean((y_hat - y) ** 2).item() * len(y)
        n += len(y)
    return mae_sum / max(n, 1), mse_sum / max(n, 1)


In [ ]:

# === CONTINUAL: circuito1 → circuito2 con EWC ===
# Leemos tasks.json para el orden de tareas
with open(PROC/"tasks.json","r",encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
task_list


In [ ]:

# Función para crear loaders de una tarea dada
def make_loader_fn(task, batch_size):
    name = task["name"]
    base = RAW/name
    paths = task["paths"]
    return make_loaders_from_csvs(
        base_dir=base,
        train_csv=Path(paths["train"]),
        val_csv=Path(paths["val"]),
        test_csv=Path(paths["test"]),
        batch_size=batch_size,
        encoder=cfg["encoder"],
        T=cfg["T"],
        gain=cfg["gain"],
        tfm=tfm
    )

# Instanciamos modelo y EWC
model2 = SNNVisionRegressor(in_channels=1, lif_beta=0.95)
ewc = EWC(model2, EWCConfig(lambd=1e10, fisher_batches=25))
loss_fn = torch.nn.MSELoss()
tcfg2 = TrainConfig(epochs=cfg["epochs"], batch_size=cfg["batch_size"], lr=cfg["lr"], amp=cfg["amp"])

outc = ROOT/"outputs"/f"continual_{preset}_ewc"
outc.mkdir(parents=True, exist_ok=True)


In [ ]:

# Bucle continual con EWC (entrena tarea, estima Fisher, evalúa test y tareas pasadas)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = {}
seen = []

for i, t in enumerate(task_list):
    name = t["name"]
    print(f"Tarea {i+1}: {name}")
    tr, va, te = make_loader_fn(t, tcfg2.batch_size)
    
    # Entrena esta tarea con EWC (el penalty se aplica dentro de train_supervised)
    _ = train_supervised(model2, tr, va, loss_fn, tcfg2, outc/f"task_{i+1}_{name}", method=ewc)
    
    # Estima Fisher sobre validación para consolidar esta tarea
    print("Estimando Fisher...")
    ewc.estimate_fisher(va, loss_fn, device=device)
    
    # Eval post-tarea
    te_mae, te_mse = eval_loader(te, model2, device)
    results[name] = {"test_mae": te_mae, "test_mse": te_mse}
    seen.append((name, te))
    
    # Evalúa tareas previas para medir olvido (BWT)
    for pname, p_loader in seen[:-1]:
        p_mae, p_mse = eval_loader(p_loader, model2, device)
        results[pname][f"after_{name}_mae"] = p_mae
        results[pname][f"after_{name}_mse"] = p_mse

# Guarda resultados
with open(outc/"continual_results.json","w",encoding="utf-8") as f:
    json.dump(results, f, indent=2)
print("OK:", outc/"continual_results.json")
results
